In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
book_filter_df=pd.read_csv('book_filter_df.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'book_filter_df.csv'

#### user-book 평점 dataframe
- user_book_df.csv

In [91]:
user_book_df=book_filter_df.pivot_table(index='user_id',columns='book_title',values='rating',fill_value=0)
book_user_df=user_book_df.T

(1924, 3374)

In [92]:
from sklearn.metrics.pairwise import cosine_similarity
book_similarity = cosine_similarity(book_user_df, book_user_df)
book_similarity_df = pd.DataFrame(book_similarity, index=user_book_df.columns, columns=user_book_df.columns)

In [93]:
book_similarity_df.to_csv("book_similarity_df.csv")

In [94]:
user_book_df.to_csv("user_book_df.csv")

#### 연령별 유사도 dataframe 
- age_similarity_df.csv
- 

In [99]:
ages_book_=book_filter_df.groupby(['ages','book_title']).size().reset_index(name='count')

In [100]:
ages_book_df=ages_book_.pivot_table(index='ages',columns='book_title',values='count',fill_value=0)
book_ages_df=ages_book_df.T

ages,10,20,30,40,50,60,70
book_title,,,,,,,
'salem's lot,0.0,0.0,4.0,1.0,0.0,0.0,0.0
10 lb. penalty,0.0,1.0,4.0,3.0,1.0,1.0,0.0
101 dalmatians,0.0,2.0,0.0,1.0,0.0,0.0,0.0
16 lighthouse road,1.0,0.0,2.0,3.0,2.0,0.0,0.0
1984,0.0,18.0,13.0,6.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...
a suitable vengeance,0.0,0.0,3.0,1.0,1.0,0.0,0.0
a superior death,0.0,0.0,3.0,1.0,3.0,0.0,0.0
a swiftly tilting planet,1.0,3.0,5.0,0.0,0.0,0.0,0.0


In [101]:
from sklearn.metrics.pairwise import cosine_similarity
ages_similarity=cosine_similarity(book_ages_df,book_ages_df)
ages_similarity_df=pd.DataFrame(ages_similarity,index=ages_book_df.columns, columns=ages_book_df.columns)
ages_similarity_df.to_csv("age_similarity_df.csv",index=True)

#### 작가유사도 dataframe
- author_ratings_pred_df.csv
- author_similarity_df.csv

In [ ]:
user_author_rating_df = book_filter_df.groupby(['user_id', 'book_author'])['rating'].mean().reset_index()
user_author_df = user_author_rating_df.pivot_table(index="user_id", columns="book_author", values="rating",fill_value=0)
author_user_df = user_author_df.T

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

user_author_sim = cosine_similarity(author_user_df, author_user_df)
user_author_sim_df = pd.DataFrame(user_author_sim, index=author_user_df.index, columns=author_user_df.index)

In [ ]:
user_author_sim_df.to_csv("author_similarity_df.csv")

In [ ]:
def predict_ratings(author_user_df, user_author_sim):
    ratings_pred = np.dot(user_author_sim, author_user_df) / np.abs(user_author_sim).sum(axis=1)[:, np.newaxis]
    return ratings_pred

ratings_pred = predict_ratings(author_user_df, user_author_sim)
ratings_pred = ratings_pred.T
ratings_pred_df = pd.DataFrame(ratings_pred, index=user_author_df.index, columns=user_author_df.columns)


In [ ]:
ratings_pred_df.to_csv("author_rating_pred_df.csv")

#### 책 평점별 유사도 dataframe
- book_ratings_pred_df.csv
- book_review_data.csv
- -book_similarity_df.csv

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
book_sim = cosine_similarity(book_user_df, book_user_df)
print(book_sim.shape)
book_sim_df = pd.DataFrame(book_sim, index=user_book_df.columns, columns=user_book_df.columns)

In [ ]:
book_sim.df.to_csv("book_similarity_df.csv")

In [ ]:
def predict_ratings(user_book_df, book_sim_df):
    return user_book_df.dot(book_sim_df) / np.abs(book_sim_df).sum(axis=1)
ratings_pred_df = predict_ratings(user_book_df, book_sim_df)

print(ratings_pred_df.shape)

In [ ]:
ratings_pred_df.to_csv("book_ratings_pred_df.csv")

### category별 추천
- top_book_by_category

In [ ]:
user_category_df = pd.DataFrame(book_filter_df[['user_id', 'book_title', 'L_Category']])
group_by_category_df = user_category_df.groupby('user_id')['L_Category'].value_counts()
group_by_category_df = group_by_category_df.reset_index()
group_by_df = user_category_df.groupby(["user_id", "L_Category", 'book_title']).size().reset_index(name='count')

# 피벗테이블로 만들기
user_category_count_df = group_by_category_df.pivot_table(index='user_id', columns='L_Category', values='count',fill_value=0)
top_category = user_category_count_df.idxmax(axis=1)

top_category_by_user_df = top_category.reset_index(name='top_L_category')

top_book_by_category = book_filter_df[['L_Category', 'book_title']].drop_duplicates()
book_ratings = book_filter_df.groupby(['L_Category', 'book_title'])["rating"].mean().reset_index()
book_filter_df['L_Category'].unique()

top_book_by_category = pd.DataFrame(index=range(1, 11), columns=book_filter_df["L_Category"].drop_duplicates().values)

columns = book_filter_df["L_Category"].drop_duplicates().values
for col in columns:
    book = book_ratings[book_ratings["L_Category"] == col].sort_values(by='rating', ascending=False)[:10]['book_title']
    top_book_by_category[col] = book.values


In [ ]:
top_book_by_category.to_csv("top_book_by_category.csv")